In [31]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""


# https://github.com/dpguthrie/yahooquery

# import pandas_datareader as pdr
import datetime as dt
import pandas as pd
from yahooquery import Ticker

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width',1000)

# Add CSV File Implementation 

In [32]:
df_stocks = pd.read_csv("Stock.csv")
df_stocks.columns = ["Stocks"]

stocklist = df_stocks["Stocks"].values.tolist()
tickers = Ticker(stocklist)


# Key Stats Data 
keystats = tickers.key_stats
esg_scores = tickers.summary_detail

In [33]:
# Storing the income statement data into a dataframe
df_balance = tickers.balance_sheet().transpose()
df_income = tickers.income_statement().transpose()
df_cashflow = tickers.cash_flow().transpose()
df_keystats = pd.DataFrame.from_dict(keystats)
df_summary = pd.DataFrame(tickers.summary_detail)

In [34]:
# Defining all the empty directories 
all_stat_income = {}
all_stat_balance = {}
all_stat_cashflow = {}
all_stat_keystat = {}
all_stat_summary = {}


# All the data directories 
stats_income = ["asOfDate", "EBIT", "NetIncome", "NetIncomeCommonStockholders"]
stats_balance = ["asOfDate", "CurrentAssets", "CurrentLiabilities", "NetPPE","StockholdersEquity","TotalDebt"]
stats_cashflow = ["asOfDate", "OperatingCashFlow", "CapitalExpenditure", "NetIncome"]
stats_keystats = ["bookValue", "enterpriseValue"]
stats_summary = ["marketCap"]

# Sorting data into one dataframe, divide up the data into sectors

for stat_income in stats_income:
    all_stat_income[stat_income] = df_income.loc[stat_income]
        
for stat_balance in stats_balance:
    all_stat_balance[stat_balance] = df_balance.loc[stat_balance]
    
for stat_cashflow in stats_cashflow: 
    all_stat_cashflow[stat_cashflow] = df_cashflow.loc[stat_cashflow]
    
for stat_keystat in stats_keystats:
    all_stat_keystat[stat_keystat] = df_keystats.loc[stat_keystat]
    
for stat_summary in stats_summary:
    all_stat_summary[stat_summary] = df_summary.loc[stat_summary]



# Adding the data into a dataframe
# final_stat_income = pd.DataFrame(all_stat_income).transpose()
# final_stat_balance = pd.DataFrame(all_stat_balance).transpose()
# final_stat_cashflow = pd.DataFrame(all_stat_cashflow).transpose()
# final_stat_keystat = pd.DataFrame(all_stat_keystat).transpose()

final_stat_income = pd.DataFrame(all_stat_income)
final_stat_balance = pd.DataFrame(all_stat_balance)
final_stat_cashflow = pd.DataFrame(all_stat_cashflow)

In [35]:
# These two dataframes do not require 
final_stat_keystat = pd.DataFrame(all_stat_keystat).sort_index()
final_stat_summary = pd.DataFrame(all_stat_summary).sort_index()

# Basically need a functionality to sort out the most recent data if possible 
# Possibly a for loop in this case??? 
# Alot of companies have different reporting dates its not possible to sort by dates 


# target_date = final_stat_income['asOfDate'][4]
# target_date_balance = final_stat_balance['asOfDate'][4]

# final_stat_income_recent = final_stat_income[final_stat_income['asOfDate'] == target_date].sort_index()
# final_stat_balance_recent = final_stat_balance[final_stat_balance['asOfDate'] == target_date_balance].sort_index()
# final_stat_cashflow_recent = final_stat_cashflow[final_stat_cashflow['asOfDate'] == target_date].sort_index()

# Create a code column for ASX codes (for groupby)
final_stat_income['code'] = final_stat_income.index
final_stat_balance['code'] = final_stat_balance.index
final_stat_cashflow['code'] = final_stat_cashflow.index

# Grabs latest date row for each ASX company
final_stat_income_recent = final_stat_income.groupby('code').tail(1)
final_stat_balance_recent = final_stat_balance.groupby('code').tail(1)
final_stat_cashflow_recent = final_stat_cashflow.groupby('code').tail(1)

In [39]:
# For loop? 

# Probably should try to combine all the data 
combine = [final_stat_income_recent, final_stat_balance_recent, final_stat_keystat, final_stat_summary]

df_combine = pd.concat(combine,axis=1)

# Calculations for the Variables, Variables required 
# TEV  - Total Enterprise Value, EBIT - Earnings Before Income Tax 
# Formula for TEV is MarketCap (Summary) + Total Debt (Balance) - Current Assets (Balance) - Current Liabilities (Balance) 
# EBIT - Can be found directly form the income statement 
# Earning Yield = EBIT/TEV
# ROC = EBIT/ PPE + Current Assets - Current Liabilities

# Create new dataframe for the calculations 
df_calc = pd.DataFrame()

df_calc["EBIT"] = df_combine["EBIT"] 
df_calc["TEV"] = df_combine["marketCap"].fillna(0) + df_combine["TotalDebt"].fillna(0) - df_combine["CurrentAssets"].fillna(0) - df_combine["CurrentLiabilities"].fillna(0)
df_calc["Net Assets and Working Capital"] = df_combine["NetPPE"].fillna(0) + df_combine["CurrentAssets"].fillna(0) - df_combine["CurrentLiabilities"].fillna(0)
df_calc = df_calc.dropna()

In [40]:
df_calc["Earnings Yield"] = df_calc["EBIT"] / df_calc["TEV"]
df_calc["Return on Capital"] = df_calc["EBIT"] / df_calc["Net Assets and Working Capital"]
df_calc["EC_Rank"] = df_calc["Earnings Yield"].rank(ascending = False, na_option = "bottom")
df_calc["ROC_Rank"] = df_calc["Return on Capital"].rank(ascending = False,na_option='bottom')
df_calc["Total"] = (df_calc["EC_Rank"] + df_calc["ROC_Rank"])
df_calc["Rank"] = df_calc["Total"].rank(method = "first")
value_stocks = df_calc.sort_values("Rank").iloc[:,[8,3,4,7]]



print(value_stocks)
x = df_calc["Earnings Yield"].mean()
print(x)


#Printing Data Into CSV File to Open on Excel
df_calc.to_csv("Calculations.csv")
value_stocks.to_csv("BlankStock.csv")

# Future Ideas Create A Model for it 
# Hypothetically Invest 10K dollars into the top 5 stocks and Track Returns
# Maybe Go Back To Previous Data
# Re adjust every 6 Months 




# Need to start ranking the calculations 

# date = ["asOfDate"]
# stat = ["EBITDA"]
# stat1 = ["DepreciationAndAmortization"]


# all_stat[0] = df_income.loc[date]
# all_stat[1] = df_income.loc[stat]
# all_stat[2] = df_cashflow.loc[stat1]

# final_df = pd.concat(all_stat.values(), ignore_index = False)



# EBIT - Income Statement
# MarketCap - Summary - Data. 
# Depreication and Ammortisation - Cash Flow 
# NetIncome - Income Statement 
# CashFlowOps - Cash Flow 
# Capex - Cashflow 
# Current Assets - Balance Sheet 
# Current Liabilities - Balance Sheet 
# PPE - Balance Sheet 
# Bookvalue - Key Stats 
# EnterpriseValue - Key Stats 

         Rank Earnings Yield Return on Capital  Total
CIM.AX    1.0       0.353962           4.22375   17.0
CTD.AX    2.0       0.276677          0.965147   40.0
MIN.AX    3.0       0.482054          0.626424   46.0
SSM.AX    4.0       0.226771           1.08969   46.0
SIQ.AX    5.0       0.139709           3.83828   51.0
NSR.AX    6.0       0.133578           2.25111   58.0
CHC.AX    7.0       0.122452           3.15108   59.0
SUN.AX    8.0        0.11369            13.375   59.0
PDL.AX    9.0       0.124438            2.2831   60.0
NEC.AX   10.0       0.240993          0.627447   63.0
PPT.AX   11.0       0.418875          0.445721   64.0
CMW.AX   12.0       0.115254           2.81638   65.0
LNK.AX   13.0       0.709849           0.39221   65.0
MGR.AX   14.0       0.109963           6.35714   66.0
SUL.AX   15.0       0.247573          0.526739   67.0
JBH.AX   16.0       0.151943          0.739693   68.0
PTM.AX   17.0       0.123158           1.07378   69.0
WOR.AX   18.0        1.97078